In [ ]:
!pip install -U transformers

In [ ]:

from transformers import ElectraModel, ElectraTokenizer

# KoELECTRA-Base
model = ElectraModel.from_pretrained("monologg/koelectra-base-discriminator")
tokenizer = ElectraTokenizer.from_pretrained("monologg/koelectra-base-discriminator")

# KoELECTRA-Small
model = ElectraModel.from_pretrained("monologg/koelectra-small-discriminator")
tokenizer = ElectraTokenizer.from_pretrained("monologg/koelectra-small-discriminator")

In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

tokenizer = AutoTokenizer.from_pretrained("Leo97/KoELECTRA-small-v3-modu-ner")
model = AutoModelForTokenClassification.from_pretrained("Leo97/KoELECTRA-small-v3-modu-ner")
ner = pipeline("ner", model=model, tokenizer=tokenizer)

example = "서울역으로 안내해줘."
ner_results = ner(example)
print(ner_results)

In [ ]:
#0. CPU 및 GPU 환경 설정

import torch
import random
import numpy as np
import torch.backends.cudnn as cudnn

# 이걸 왜 하는거임 ?
torch.manual_seed(42)

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [ ]:
torch.cuda.get_device_name(0)

### 한국해양대학교 자연어처리 연구실 NER 데이터셋

In [ ]:
!git clone https://github.com/kmounlp/NER.git

In [ ]:
import os
import glob

In [ ]:
file_list = []

In [ ]:
for x in os.walk('NER/'):
    for y in glob.glob(os.path.join(x[0], '*_NER.txt')):
        file_list.append(y)

In [ ]:
file_list = sorted(file_list)

In [ ]:
for file_path in file_list:
    print(file_path)

In [ ]:
# 한국 해양대학교 데이터셋
from pathlib import Path

file_path = file_list[0]
file_path = Path(file_path)
raw_text = file_path.read_text().strip()

In [ ]:
print(raw_text[0:1000])

### NAVER NLP Challenge 2018 NER 데이터셋


In [ ]:
!pip install korpora

In [ ]:
from Korpora import Korpora
corpus = Korpora.load("naver_changwon_ner")

In [ ]:
from google.colab import drive

drive.mount('/content/drive', force_remount=True)

%cd 'drive/MyDrive/공적말하기_프로젝트/02. 프로젝트 실행/jwjang_workspace/태그인식 테스트/dataset_koelectra'
# /MyDrive/공적말하기_프로젝트/02. 프로젝트 실행/jwjang_workspace/태그인식 테스트/dataset_koelectra
!ls

In [ ]:
import pandas as pd

org = pd.read_csv('상장법인목록.csv',)
org_df = org.iloc[:, 0:1]

#### 네이버 데이터셋 전처리

In [ ]:
def naver_read_file(file_list):

  token_docs = []
  tag_docs = []

  for doc in file_list:
    tokens = []
    tags = []
    list1=doc.words
    list2=doc.tags

    for text, docs in zip(list1, list2):
      try:
        tag = docs
        if tag == 'ORG-B':
          tag='B-ORG'

        # 2: pos, 3: ner
          if tag in 'B-ORG':
            token = random.sample(org_df['회사명'].tolist(), k=1)[0] # tag가 B-ORG이면 상장법인목록의 회사명 중 하나로 랜덤으로
          else:
            token = text
        else:
          token = text
          tag = 'O'
        for i, syllable in enumerate(token): # 음절 단위로 자르고
          tokens.append(syllable)
          modi_tag = tag
          if i > 0:
            if tag[0] == 'B':
              modi_tag = 'I' + tag[1:] # BIO tag를 부착
          tags.append(modi_tag)
      except:
        continue
    token_docs.append(tokens)
    tag_docs.append(tags)

  return token_docs, tag_docs


In [ ]:
naver_text, naver_tags = naver_read_file(corpus.train)

In [ ]:
print(len(naver_text))
print(len(naver_tags))

In [ ]:
print(naver_text[0], end='\n\n') # 음절 단위로 잘 잘림
print(naver_tags[0])

In [ ]:
import re
def read_file(file_list):
  token_docs = []
  tag_docs = []
  for file_path in file_list:
    # print('read file from ', file_path)
    file_path = Path(file_path)
    raw_text = file_path.read_text().strip()
    raw_docs = re.split(r'\n\t?\n', raw_text)
    for doc in raw_docs:
      tokens = []
      tags = []
      for line in doc.split('\n'):
        if line[0:1] == "$" or line[0:1] == ";" or line[0:2] == "##":
          continue
        try:
          tag = line.split('\t')[3] # 2: pos, 3: ner
          if tag in ['B-ORG', 'I-ORG']:
            if tag == 'B-ORG':
              token = random.sample(org_df['회사명'].tolist(), k=1)[0]
            elif tag == 'I-ORG':
              token = None
            else:
              toke = line.split('\t')[0]
          else:
            token = line.split('\t')[0]
            tag = 'O'
          for i, syllable in enumerate(token): # 음절 단위로 잘라서
            tokens.append(syllable)
            modi_tag = tag
            if i > 0:
              if tag[0] == 'B':
                modi_tag = 'I' + tag[1:] # BIO tag를 부착
            tags.append(modi_tag)
        except:
          continue
      token_docs.append(tokens)
      tag_docs.append(tags)

    return token_docs, tag_docs


In [ ]:
texts, tags = read_file(file_list[:])

In [ ]:
import urllib.request
import pandas as pd

# 1. 데이터셋 준비하기
urllib.request.urlretrieve("https://raw.githubusercontent.com/bab2min/corpus/master/sentiment/naver_shopping.txt", filename="ratings_total.txt")

# 2. 모델 구성하기
# 3. 모델 학습과정 설정하기
# 4. 모델 학습시키기
# 5. 모델 학습과정 살펴보기
# 6. 모델 평가하기
# 7. 모델 사용하기

In [ ]:
total_data = pd.read_table('ratings_total.txt', names=['ratings', 'reviews'])
print('전체 리뷰 개수 :',len(total_data)) # 전체 리뷰 개수 출력

In [ ]:
total_data[:5]